#### Feasible Circulation Problem: Reduction to Max-Flow

In this problem, we have a `circulation network` $G = (V,E)$ containing a set of multiple sources $S \subset E $ and a set of sinks $T \subset E$. Each vertex $v \in V$ has a `demand value` $d_v$ associated with it. The goal is to find a `feasible circulation`, i.e. a flow $f$ for which $f^{in}(v)-f^{out}(v)=d_v$ for all $v$. 

This problem can be solved by reduction of the circulation network $G$ into a flow network $G'=(V',E')$ where 

$V' = V + \{s^*,t^*\}$, 

$E'= E + \{(s^*,v)| \forall v \in S\}+ \{(v,t^*)| \forall v \in T\}$,

capacity of each edge $(s^*,v)$ is $-d_v$ and capacity of each edge $(v,t^*)$ is $d_v$. 

We've introduced two new nodes, $s^*$ which is a single `super-source` and $t^*$ is a `super-sink`. Then if we solve for the max-flow $f$ in $G'$ and find that $|f|=D=\sum_{v\in S} -d_v = \sum_{v\in T} d_v$, then f is a feasible circulation in $G$.    

We now implement this algorithm.

In [19]:
# use BFS to find path from s to t in Gf
def find_st_path(adjacency_list, s, t):
    assert (s in adjacency_list and t in adjacency_list), "s and t must be vertices in the graph!"
    # initialize all vertices as undiscovered except for s
    discovered = {u:False for u in adjacency_list.keys()}
    discovered[s] = True
    parent = {u:None for u in adjacency_list.keys()}
    # intiialize BFS queue
    Q = []
    Q.append(s)
    # initialize s-t path
    path = []
    # run BFS
    while len(Q)>0:
        # pop vertex from front of queue
        u = Q.pop(0)
        # find vertices adjacent to u and add them to the Q if they are undiscovered
        for (v,ce) in adjacency_list[u]:
            if not discovered[v]:
                discovered[v] = True
                parent[v] = (u, ce)
                # if v is t, then we have found the path
                if v == t:
                    # reconstruct path from s to t following parent pointers
                    while parent[t] != None:
                        # add edge to path
                        edge = (parent[t][0], t, parent[t][1])
                        path.append(edge)
                        t = parent[t][0]
                    return path
                # add to Q
                Q.append(v)
 
    # if we reach here, then there is no s-t path
    return path            


def construct_residual_graph(adjacency_list, flow, capacity):
    Gf_adjacency_list = {u:[] for u in adjacency_list.keys()}
    for u in adjacency_list.keys():
        for (v, _) in adjacency_list[u]:
            # add forward edge if there's non-zero residual capacity
            residual_ce = capacity[(u, v)] - flow[(u, v)]
            if residual_ce > 0:
                Gf_adjacency_list[u].append((v, residual_ce))
            # add backward edge if there's non-zero flow 
            if flow[(u, v)] > 0:
                Gf_adjacency_list[v].append((u, flow[(u, v)]))

    return Gf_adjacency_list            

def FF_nosclaing(adjacency_list, s, t, verbose=False):
    # initialize the flow to 0
    flow = {}
    capacity = {}
    for u in adjacency_list.keys():
        for (v, ce) in adjacency_list[u]:
            flow[(u, v)] = 0
            capacity[(u, v)] = ce

    # construct residual graph
    Gf_adjacency_list = construct_residual_graph(adjacency_list, flow, capacity)
    if verbose:
        print("Initial residual graph:")
        print(Gf_adjacency_list)
 
    num_iterations = 0
    # run augmenting iterations
    while True:
        # find s-t path in Gf
        path = find_st_path(Gf_adjacency_list, s, t)
        # if there is no s-t path, then we are done
        if len(path) == 0:
            if verbose:
                flow_value = sum([flow[(s, v)] for (v, _) in adjacency_list[s]])
                print(f"\nMax flow found! |f| = {flow_value}, total iterations = {num_iterations}")
            return flow, flow_value
        else:
            # find bottleneck capacity of path
            bottleneck_capacity = float('inf')
            for (u, v, ce) in path:
                if ce < bottleneck_capacity:
                    bottleneck_capacity = ce
            # augment flow along path
            for (u, v, ce) in path:
                if (u, v) in capacity:
                    # increase flow along forward edge (u, v)
                    flow[(u, v)] += bottleneck_capacity
                else:
                    # decrease flow along backward edge (v, u)
                    flow[(v, u)] -= bottleneck_capacity 
            # construct updated residual graph        
            Gf_adjacency_list = construct_residual_graph(adjacency_list, flow, capacity)     
            
            if verbose:
                print(f"\nFlow augmented along path: {path} by amount: {bottleneck_capacity}")
                print(f"New residual graph: \n {Gf_adjacency_list}")
                print(f"Augmented Flow: \n {flow}")   

            num_iterations += 1        
            

def reduced_flow_network(G, demand):
    # get source vertices
    sources = [v for v in G.keys() if demand[v] < 0]
    # get sink vertices
    sinks = [v for v in G.keys() if demand[v] > 0]
    D = sum([demand[v] for v in sinks])
    # create adjacency list of recuced flow network G' 
    super_source = 's'
    super_sink = 't'
    G_prime = {u:list(G[u]) for u in G.keys()}
    G_prime[super_source] = [(v, -demand[v]) for v in sources]
    G_prime[super_sink] = []
    for v in sinks:
        G_prime[v].append((super_sink, demand[v]))
    
    return G_prime, super_source, super_sink, D

def feasible_circulation(adjacency_list, demand, verbose=True):
    # construct redeuced flow network
    G_prime, super_source, super_sink, D = reduced_flow_network(adjacency_list, demand)
    if verbose:
        print(f"Reduced flow network G': {G_prime}")
        print(f"Super source: {super_source}, Super sink: {super_sink}, Total demand: {D}")

    # run FF algorithm on G'
    f, flow_value = FF_nosclaing(G_prime, super_source, super_sink, verbose=verbose)

    feasible_f = None
    # check if flow is feasible
    if flow_value == D:
        feasible_f = {(u,v):fe for (u,v),fe in f.items() if u!=super_source and v!=super_sink}
        if verbose:
            print(f"\nFeasible circulation found! |f| = {flow_value}")
            print(f"Feasible circulation: {feasible_f}")

    return feasible_f                        

Example circulation network $G$ on the left and reduced flow network $  G'$ on the right. The flow values are inside the square boxes and number inside each verted is $d_v$

<img src="circulation_figure.png" width="600" height="230">

In [20]:
# adjacenecy list of each node contains a tuple of (node, capacity)
adjacency_list = {'a':[('c',2), ('d',2)], 'b':[('a',3), ('d',3)], 'c':[], 'd':[('c',2)]}
demand = {'a':-3, 'b':-3, 'c':4, 'd':2}

# find feasible circulation
feasible_f = feasible_circulation(adjacency_list, demand)

Reduced flow network G': {'a': [('c', 2), ('d', 2)], 'b': [('a', 3), ('d', 3)], 'c': [('t', 4)], 'd': [('c', 2), ('t', 2)], 's': [('a', 3), ('b', 3)], 't': []}
Super source: s, Super sink: t, Total demand: 6
Initial residual graph:
{'a': [('c', 2), ('d', 2)], 'b': [('a', 3), ('d', 3)], 'c': [('t', 4)], 'd': [('c', 2), ('t', 2)], 's': [('a', 3), ('b', 3)], 't': []}

Flow augmented along path: [('c', 't', 4), ('a', 'c', 2), ('s', 'a', 3)] by amount: 2
New residual graph: 
 {'a': [('d', 2), ('s', 2)], 'b': [('a', 3), ('d', 3)], 'c': [('a', 2), ('t', 2)], 'd': [('c', 2), ('t', 2)], 's': [('a', 1), ('b', 3)], 't': [('c', 2)]}
Augmented Flow: 
 {('a', 'c'): 2, ('a', 'd'): 0, ('b', 'a'): 0, ('b', 'd'): 0, ('c', 't'): 2, ('d', 'c'): 0, ('d', 't'): 0, ('s', 'a'): 2, ('s', 'b'): 0}

Flow augmented along path: [('d', 't', 2), ('a', 'd', 2), ('s', 'a', 1)] by amount: 1
New residual graph: 
 {'a': [('d', 1), ('s', 3)], 'b': [('a', 3), ('d', 3)], 'c': [('a', 2), ('t', 2)], 'd': [('a', 1), ('c', 2), 